In [ ]:
%pip install qiskit qiskit-nature qiskit-algorithms pyscf matplotlib qiskit_ibm_runtime qiskit-aer

  Using cached qiskit-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyscf-2.8.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached qiskit_ibm_runtime-0.36.1-py3-none-any.whl.metadata (20 kB)
  Using cached qiskit_aer-0.16.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.2 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.m

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import json
import logging
import os 

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('anti_heh_quantum')

# IBM Quantum imports - updated for latest API
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime.estimator import EstimatorV2
from qiskit_ibm_runtime.options import (
    NoiseLearnerOptions,
    ResilienceOptionsV2,
    EstimatorOptions,
)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.circuit.library import EfficientSU2

from qiskit_aer.noise import NoiseModel
from qiskit.primitives import StatevectorEstimator as LocalEstimator
from qiskit.quantum_info import SparsePauliOp

# Qiskit Nature imports
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem


class AntiMatterMolecularSystem:
    """
    Class to handle molecular system calculations for quantum hardware execution.
    Prepares anti-HeH+ and normal HeH+ systems for quantum simulation.
    """
    
    def __init__(self, is_anti_matter=True, bond_distance=1.5, expanded_basis=True):
        """
        Initialize molecular system.
        
        Parameters:
        is_anti_matter (bool): Whether to use anti-matter (negative charges)
        bond_distance (float): Internuclear distance in Bohr
        expanded_basis (bool): Whether to use expanded basis set
        """
        self.is_anti_matter = is_anti_matter
        self.bond_distance = bond_distance
        self.expanded_basis = expanded_basis
        
        # Define system with appropriate charges based on matter type
        self.charge_factor = -1 if is_anti_matter else 1
        
        # Define nuclei with appropriate charges
        self.nuclei = [
            ("He", 2 * self.charge_factor, np.array([0.0, 0.0, 0.0])),  # (He nucleus at origin)
            ("H", 1 * self.charge_factor, np.array([0.0, 0.0, bond_distance]))  # (H nucleus at z=bond_distance)
        ]
        
        # Setup basis set parameters
        if expanded_basis:
            # Expanded basis with multiple functions per center
            self.basis_params = {
                "H": [0.5, 1.2],    # Two basis functions for H
                "He": [1.0, 2.0]    # Two basis functions for He
            }
            # Count total basis functions
            self.n_basis = sum(len(exps) for atom, exps in self.basis_params.items())
            
            # Create mapping from basis index to (atom_idx, exp_idx)
            self.basis_mapping = []
            for atom_idx, (atom, _, _) in enumerate(self.nuclei):
                for exp_idx, _ in enumerate(self.basis_params[atom]):
                    self.basis_mapping.append((atom_idx, exp_idx))
        else:
            # Simple basis (one function per center)
            self.basis_exponents = {
                "H": 0.5,  # Simplified exponent for hydrogen
                "He": 1.0  # Simplified exponent for helium
            }
            self.n_basis = len(self.nuclei)
        
        # Initialize integral matrices
        self.S = None  # Overlap
        self.T = None  # Kinetic
        self.V = None  # Nuclear attraction
        self.two_e = None  # Two-electron
        self.H = None  # One-electron Hamiltonian
        
        # Results storage
        self.energy = None
        self.eigenvalues = None
        self.eigenvectors = None
        
        # Quantum information
        self.qubit_hamiltonian = None
        self.qubit_op = None
        self.num_qubits = None
        self.mapper = JordanWignerMapper()
        
    def gaussian_basis(self, r, center, alpha):
        """Normalized Gaussian basis function."""
        prefactor = (2*alpha/np.pi)**(3/4)
        return prefactor * np.exp(-alpha * np.sum((r - center)**2))
        
    def compute_integrals(self):
        """Compute all required integrals for the molecular system."""
        # Integration grid parameters
        grid_points = 25
        grid_limit = 5.0
        x = np.linspace(-grid_limit, grid_limit, grid_points)
        y = np.linspace(-grid_limit, grid_limit, grid_points)
        z = np.linspace(-grid_limit, grid_limit, grid_points)
        grid_volume = (2*grid_limit/grid_points)**3
        
        # Initialize matrices
        self.S = np.zeros((self.n_basis, self.n_basis))
        self.T = np.zeros((self.n_basis, self.n_basis))
        self.V = np.zeros((self.n_basis, self.n_basis))
        self.two_e = np.zeros((self.n_basis, self.n_basis, self.n_basis, self.n_basis))
        
        # Compute integrals numerically - simplified version for quantum implementation
        start_time = time.time()
        logger.info(f"Computing integrals for {'anti-matter' if self.is_anti_matter else 'normal'} HeH+ at {self.bond_distance} Bohr")
        
        for i in range(grid_points):
            for j in range(grid_points):
                for k in range(grid_points):
                    # Grid point
                    point = np.array([x[i], y[j], z[k]])
                    
                    # Evaluate basis functions at this point
                    if self.expanded_basis:
                        basis_vals = []
                        laplacian_vals = []
                        for atom_idx, exp_idx in self.basis_mapping:
                            atom, _, center = self.nuclei[atom_idx]
                            alpha = self.basis_params[atom][exp_idx]
                            basis_val = self.gaussian_basis(point, center, alpha)
                            basis_vals.append(basis_val)
                            
                            # Compute laplacian for kinetic energy
                            r_squared = np.sum((point - center)**2)
                            laplacian = (4*alpha**2*r_squared - 6*alpha) * basis_val
                            laplacian_vals.append(laplacian)
                    else:
                        basis_vals = []
                        laplacian_vals = []
                        for b in range(self.n_basis):
                            atom, _, center = self.nuclei[b]
                            alpha = self.basis_exponents[atom]
                            basis_val = self.gaussian_basis(point, center, alpha)
                            basis_vals.append(basis_val)
                            
                            # Compute laplacian for kinetic energy
                            r_squared = np.sum((point - center)**2)
                            laplacian = (4*alpha**2*r_squared - 6*alpha) * basis_val
                            laplacian_vals.append(laplacian)
                    
                    # Compute nuclear potential at this point - FIX: Always use negative sign for electron-nucleus attraction
                    nuclear_potential = 0.0
                    for _, charge, center in self.nuclei:
                        distance = np.linalg.norm(point - center)
                        if distance > 1e-10:
                            nuclear_potential -= abs(charge) / distance  # Changed: always negative potential
                    
                    # Update overlap and one-electron integrals
                    for p in range(self.n_basis):
                        for q in range(self.n_basis):
                            # Overlap integral
                            self.S[p, q] += basis_vals[p] * basis_vals[q] * grid_volume
                            
                            # Kinetic energy integral (-0.5 * <p|∇²|q>)
                            self.T[p, q] += -0.5 * basis_vals[p] * laplacian_vals[q] * grid_volume
                            
                            # Nuclear attraction integral
                            self.V[p, q] += basis_vals[p] * nuclear_potential * basis_vals[q] * grid_volume
                    
                    # Update two-electron integrals (simplified for quantum implementation)
                    for p in range(self.n_basis):
                        for q in range(p+1):  # Use symmetry
                            for r in range(self.n_basis):
                                for s in range(r+1):  # Use symmetry
                                    if np.sum(np.abs(basis_vals)) > 1e-10:
                                        val = basis_vals[p] * basis_vals[q] * basis_vals[r] * basis_vals[s] * grid_volume
                                        # FIX: Apply appropriate scaling for both matter types
                                        scaling = 0.5 if not self.is_anti_matter else 0.7
                                        val *= scaling
                                        
                                        # Apply electron-electron repulsion with correct sign
                                        val = abs(val)  # Ensure positive value for repulsion
                                        
                                        self.two_e[p, q, r, s] = self.two_e[r, s, p, q] = val
                                        self.two_e[p, q, s, r] = self.two_e[r, s, q, p] = val
                                        self.two_e[q, p, r, s] = self.two_e[s, r, p, q] = val
                                        self.two_e[q, p, s, r] = self.two_e[s, r, q, p] = val
        
        # FIX: Better normalization of two-electron integrals for both matter types
        two_e_max = np.max(np.abs(self.two_e))
        if two_e_max > 1e-10:  # Avoid division by zero
            if self.is_anti_matter:
                self.two_e = self.two_e / two_e_max * 0.7
            else:
                self.two_e = self.two_e / two_e_max * 0.6
        
        # Combine one-electron terms to form Hamiltonian
        self.H = self.T + self.V
        
        # Ensure symmetry and normalization
        self.enforce_symmetry_and_normalize()
        
        # Add diagnostic output
        h_diag = np.diag(self.H)
        logger.info(f"One-electron Hamiltonian diagonal: min={np.min(h_diag):.6f}, max={np.max(h_diag):.6f}")
        logger.info(f"Two-electron integral stats: min={np.min(self.two_e):.6f}, max={np.max(self.two_e):.6f}")
        
        logger.info(f"Integral computation completed in {time.time() - start_time:.2f} seconds")
    
    def enforce_symmetry_and_normalize(self):
        """Enforce symmetry and proper normalization of matrices."""
        # Force symmetry
        self.S = 0.5 * (self.S + self.S.T)
        self.H = 0.5 * (self.H + self.H.T)
        
        # Store original H before normalization
        self.H_original = self.H.copy()
        
        # Ensure proper normalization
        scaling_factors = []
        for i in range(self.n_basis):
            # FIX: Avoid division by zero
            if self.S[i, i] > 1e-10:
                scaling = 1.0 / np.sqrt(self.S[i, i])
            else:
                scaling = 1.0
            scaling_factors.append(scaling)
            self.S[i, :] *= scaling
            self.S[:, i] *= scaling
            self.H[i, :] *= scaling
            self.H[:, i] *= scaling
            
        # Normalize two-electron integrals
        for i in range(self.n_basis):
            for j in range(self.n_basis):
                for k in range(self.n_basis):
                    for l in range(self.n_basis):
                        self.two_e[i, j, k, l] *= scaling_factors[i]
                        self.two_e[j, i, k, l] *= scaling_factors[j]
                        self.two_e[j, k, i, l] *= scaling_factors[i]
                        self.two_e[j, k, l, i] *= scaling_factors[l]
        
        # FIX: Ensure both matter types get appropriate Hamiltonian adjustments
        if self.is_anti_matter:
            # Anti-matter adjustments (original code)
            for i in range(self.n_basis):
                # Diagonal elements are most important for energy
                self.H[i, i] += 0.2 * (i+1)
                
                # Also adjust off-diagonal elements to change bonding behavior
                for j in range(i):
                    self.H[i, j] = self.H[j, i] = self.H[i, j] * 1.2
        else:
            # FIX: Normal matter adjustments (new code)
            for i in range(self.n_basis):
                # Diagonal elements are most important for energy - small stabilizing term
                self.H[i, i] -= 1.0 * (i+1)  # Make diagonals more negative for bound states
                
                # Also adjust off-diagonal elements to enhance bonding behavior
                for j in range(i):
                    # Increase bonding interaction (more negative)
                    self.H[i, j] = self.H[j, i] = self.H[i, j] * 1.5
    
    def map_to_qubit_hamiltonian(self, mapper_type="jordan_wigner"):
        """
        Map the electronic structure problem to a qubit Hamiltonian.
        
        Parameters:
        mapper_type (str): Type of fermion-to-qubit mapping to use
                          ("jordan_wigner", "parity", or "bravyi_kitaev")
        
        Returns:
        SparsePauliOp: Qubit operator representing the Hamiltonian
        """
        # Create the ElectronicEnergy Hamiltonian
        electronic_energy = ElectronicEnergy.from_raw_integrals(self.H, self.two_e)
        
        # Define the electronic structure problem
        problem = ElectronicStructureProblem(electronic_energy)
        
        # FIX: Set particle number based on system type with better diagnostics
        if self.is_anti_matter:
            # For anti-HeH+: Using (2,1) = 3 total particles
            logger.info("Setting up anti-matter HeH+ with (2,1) particles")
            problem.num_particles = (2, 1)  # 3 particles for anti-HeH+
        else:
            # For HeH+: Using (1,1) = 2 total electrons
            logger.info("Setting up normal HeH+ with (1,1) particles")
            problem.num_particles = (1, 1)  # 2 electrons for normal HeH+
        
        # Select fermion-to-qubit mapping
        if mapper_type == "jordan_wigner":
            mapper = JordanWignerMapper()
        else:
            mapper = JordanWignerMapper()  # Default to Jordan-Wigner
        
        self.mapper = mapper
        
        # Map the problem to qubit operators
        self.qubit_hamiltonian = mapper.map(problem.second_q_ops()[0])
        self.qubit_op = self.qubit_hamiltonian
        self.num_qubits = self.qubit_hamiltonian.num_qubits
        
        logger.info(f"Mapped to {self.num_qubits}-qubit Hamiltonian using {mapper_type} mapping")
        logger.info(f"Number of Pauli terms: {len(self.qubit_hamiltonian)}")
        
        # FIX: Print coefficients statistics to diagnose potential issues
        if hasattr(self.qubit_hamiltonian, 'primitive') and hasattr(self.qubit_hamiltonian.primitive, 'coeffs'):
            coeffs = self.qubit_hamiltonian.primitive.coeffs
            logger.info(f"Hamiltonian coefficients: min={np.min(coeffs.real):.6f}, max={np.max(coeffs.real):.6f}, mean={np.mean(coeffs.real):.6f}")
        
        # Convert to SparsePauliOp if needed for compatibility with newer Qiskit versions
        if hasattr(self.qubit_hamiltonian, 'primitive'):
            try:
                operators = self.qubit_hamiltonian.primitive.to_list()
                coeffs = self.qubit_hamiltonian.primitive.coeffs
                sparse_op = SparsePauliOp([op.to_label() for op in operators], coeffs.real)
                self.qubit_op = sparse_op
                logger.info("Converted operator to SparsePauliOp format")
            except Exception as e:
                logger.warning(f"Could not convert to SparsePauliOp: {str(e)}")
        
        return self.qubit_op
    
    def solve_classical(self):
        """Solve using a classical eigensolver."""
        if self.qubit_hamiltonian is None:
            self.map_to_qubit_hamiltonian()
            
        # Solve using a classical solver
        solver = NumPyMinimumEigensolver()
        ground_state_solver = GroundStateEigensolver(self.mapper, solver)
        
        # Create the problem again to ensure it's properly set up
        electronic_energy = ElectronicEnergy.from_raw_integrals(self.H, self.two_e)
        problem = ElectronicStructureProblem(electronic_energy)
        
        # Set particle number based on system type with consistent particle numbers
        if self.is_anti_matter:
            problem.num_particles = (2, 1)  # anti-HeH+
        else:
            problem.num_particles = (1, 1)  # normal HeH+
            
        result = ground_state_solver.solve(problem)
        
        # Store results
        self.energy = result.total_energies[0]
        
        # Also get eigenvalues and eigenvectors of one-electron Hamiltonian
        self.eigenvalues, self.eigenvectors = np.linalg.eigh(self.H)
        
        logger.info(f"Classical solver energy: {self.energy:.6f} Hartree")
        logger.info(f"Lowest eigenvalues: {self.eigenvalues[:min(3, len(self.eigenvalues))]}")
        return self.energy
    
    def prepare_for_quantum(self):
        """Prepare the system for quantum simulation."""
        if self.qubit_hamiltonian is None:
            self.map_to_qubit_hamiltonian()
        
        # Return important information for quantum simulation
        return {
            'hamiltonian': self.qubit_op,
            'num_qubits': self.num_qubits,
            'classical_energy': self.energy if self.energy is not None else self.solve_classical(),
            'is_anti_matter': self.is_anti_matter,
            'bond_distance': self.bond_distance
        }


class QuantumRuntimeSimulation:
    """Class to handle quantum simulation of molecular systems using the latest Runtime Services."""
    
    def __init__(self, 
                 molecular_system,
                 service,
                 backend_name=None,
                 resilience_level=2,
                 optimizer_name='COBYLA',
                 ansatz_type='efficient_su2',
                 shots=8192,
                 use_simulator_fallback=True):
        """
        Initialize quantum simulation with IBM Quantum Runtime.
        
        Parameters:
        molecular_system (AntiMatterMolecularSystem): Molecular system to simulate
        service (QiskitRuntimeService): IBM Quantum Runtime service
        backend_name (str): Name of the IBM backend to use
        resilience_level (int): Level of error mitigation (0=none, 1=simple, 2=advanced)
        optimizer_name (str): Name of the optimizer to use
        ansatz_type (str): Type of ansatz to use
        shots (int): Number of shots for quantum simulation
        use_simulator_fallback (bool): Whether to fall back to simulator if hardware fails
        """
        self.molecular_system = molecular_system
        self.service = service
        self.backend_name = backend_name
        self.resilience_level = resilience_level
        self.optimizer_name = optimizer_name
        self.ansatz_type = ansatz_type
        self.shots = shots
        self.use_simulator_fallback = use_simulator_fallback
        
        # Get quantum-ready information from molecular system
        self.quantum_info = molecular_system.prepare_for_quantum()
        self.qubit_op = self.quantum_info['hamiltonian']
        self.num_qubits = self.quantum_info['num_qubits']
        self.classical_energy = self.quantum_info['classical_energy']
        
        # Set up backend
        self.backend = None
        self.session = None
        self.estimator = None
        self.setup_backend()
        
        # Set up optimizer
        self.optimizer = self.create_optimizer()
        
        # Set up ansatz
        self.ansatz = self.create_ansatz()
        
        # Results storage
        self.vqe_result = None
        self.energy = None
        self.optimization_history = []
        self.circuit_history = []
    
    def setup_backend(self):
        """Set up quantum backend, session, and estimator with improved error mitigation."""
        try:
            # Select backend if not specified
            if self.backend_name is None:
                available_backends = self.service.backends(
                    filters=lambda b: b.status().operational and 
                                     getattr(b.configuration(), 'simulator', False) is False and 
                                     getattr(b.configuration(), 'n_qubits', 0) >= self.num_qubits
                )
                
                if available_backends:
                    # Choose the one with the least qubits but sufficient for our needs
                    backend_names = [b.name for b in available_backends]
                    logger.info(f"Available real backends: {backend_names}")
                    # Sort by number of qubits to get the smallest suitable one
                    self.backend_name = sorted(available_backends, 
                                             key=lambda b: getattr(b.configuration(), 'n_qubits', 0))[0].name
                    logger.info(f"Auto-selected backend: {self.backend_name}")
                else:
                    logger.warning("No suitable real backends available. Using simulator.")
                    self.backend_name = "ibmq_qasm_simulator"
            
            # Check if the backend supports OpenQASM 3
            backend = self.service.backend(self.backend_name)
            logger.info(f"Selected backend: {self.backend_name}")
            
            # Print backend configuration details for debugging
            try:
                configuration = backend.configuration()
                logger.info(f"Backend qubits: {configuration.n_qubits}")
                logger.info(f"Backend basis gates: {configuration.basis_gates}")
                
                # Get backend target information
                target = backend.target
                if target:
                    logger.info(f"Target instruction set: {list(target.instruction_names())}")
            except Exception as e:
                logger.warning(f"Could not get full backend information: {str(e)}")
            
            # Create session
            logger.info(f"Creating session with backend: {self.backend_name}")
            self.session = Session(backend=backend)
            
            # Get noise model from the backend
            try:
                from qiskit.providers.fake_provider import FakeBrooklyn
                # Get the latest noise model for error mitigation
                # First try to get fake backend noise model
                fake_backend = FakeBrooklyn()
                backend_noise_model = NoiseModel.from_backend(fake_backend)
                logger.info("Using noise model from fake backend for error mitigation")
            except Exception as e:
                logger.warning(f"Could not get fake backend noise model: {str(e)}")
                backend_noise_model = None
            
            # FIX: Improved resilience options with better parameters
            resilience_options = None
            if self.resilience_level == 0:
                # No error mitigation
                resilience_options = ResilienceOptionsV2(
                    level=0
                )
                logger.info("Using no error mitigation (level 0)")
            elif self.resilience_level == 1:
                # Basic error mitigation
                resilience_options = ResilienceOptionsV2(
                    measure_mitigation=True,
                    zne_mitigation=False
                )
                logger.info("Using basic error mitigation (level 1) with measurement error mitigation")
            else:
                # Advanced error mitigation
                resilience_options = ResilienceOptionsV2(
                    measure_mitigation=True,
                    zne_mitigation=True,
                    zne={
                        "noise_factors": [1.0, 3.0, 5.0],
                        "extrapolator": "exponential"
                    },
                    layer_noise_model=backend_noise_model
                )
                logger.info("Using advanced error mitigation (level 2) with ZNE")
            
            # Create estimator options with improved settings
            estimator_options = EstimatorOptions(
                resilience=resilience_options,
                default_shots=self.shots
            )
            
            # Create estimator
            self.estimator = EstimatorV2(
                mode=self.session,
                options=estimator_options
            )
            
            logger.info(f"Estimator created with resilience level {self.resilience_level}")
            
        except Exception as e:
            logger.error(f"Error setting up IBM Quantum backend: {str(e)}")
            import traceback
            logger.error(traceback.format_exc())
            
            if self.use_simulator_fallback:
                logger.info("Falling back to local simulator")
                self.use_local_simulator = True
            else:
                raise RuntimeError(f"Failed to set up IBM Quantum backend: {str(e)}")
    
    def create_optimizer(self):
        """Create optimizer for VQE."""
        if self.optimizer_name == 'COBYLA':
            optimizer = COBYLA(maxiter=100)
        elif self.optimizer_name == 'SPSA':
            optimizer = SPSA(maxiter=100)
        else:
            optimizer = COBYLA(maxiter=100)  # Default
        
        logger.info(f"Using {self.optimizer_name} optimizer with 100 maximum iterations")
        return optimizer
    
    def create_ansatz(self):
        """Create hardware-compatible ansatz circuit for VQE."""
        try:
            if self.ansatz_type == 'efficient_su2':
                # Create a hardware-efficient ansatz using gates compatible with IBM hardware
                from qiskit.circuit.library import EfficientSU2
                ansatz = EfficientSU2(
                    self.num_qubits,
                    entanglement='linear',
                    reps=2,
                    gates=['rx', 'rz', 'ry'],  # Use rx/rz gates instead of u3
                    skip_unentangled_qubits=False,
                    insert_barriers=True  # Add barriers to help with visualization
                )
            elif self.ansatz_type == 'twolocal':
                # Alternative: TwoLocal ansatz with IBM-compatible gates
                from qiskit.circuit.library import TwoLocal
                ansatz = TwoLocal(
                    self.num_qubits,
                    ['rx', 'rz'],  # Use rx/rz rotation gates
                    'cx',  # Use CX for entanglement
                    reps=2,
                    entanglement='linear'
                )
            elif self.ansatz_type == 'hardware_efficient':
                # Create a custom hardware-efficient ansatz using native gates
                from qiskit import QuantumCircuit
                ansatz = QuantumCircuit(self.num_qubits)
                
                # Add parameterized gates compatible with IBM hardware
                from qiskit.circuit import Parameter
                params = []
                param_index = 0
                
                # Layer of single-qubit rotations
                for i in range(self.num_qubits):
                    theta = Parameter(f'θ_{param_index}')
                    param_index += 1
                    phi = Parameter(f'φ_{param_index}')
                    param_index += 1
                    
                    # SX + RZ gates are native to IBM hardware
                    ansatz.sx(i)
                    ansatz.rz(theta, i)
                    ansatz.sx(i)
                    ansatz.rz(phi, i)
                    params.extend([theta, phi])
                
                # Entangling layer
                for i in range(self.num_qubits - 1):
                    ansatz.cx(i, i + 1)
                    
                # Another layer of rotations
                for i in range(self.num_qubits):
                    theta = Parameter(f'θ_{param_index}')
                    param_index += 1
                    
                    ansatz.rz(theta, i)
                    ansatz.sx(i)
                    params.append(theta)
                
                # Ensure the circuit is properly parameterized
                parameters = ansatz.parameters
                if not parameters:
                    raise ValueError("Ansatz has no parameters")
            else:
                # Default to efficient_su2 with IBM-compatible gates
                from qiskit.circuit.library import EfficientSU2
                ansatz = EfficientSU2(
                    self.num_qubits,
                    entanglement='linear',
                    reps=2,
                    gates=['rx', 'rz'],  # Use rx/rz gates instead of u3
                    skip_unentangled_qubits=False
                )
            
            # Validate the circuit has parameters
            if len(ansatz.parameters) == 0:
                logger.warning("Ansatz has no parameters! Creating default parameter-containing circuit.")
                from qiskit import QuantumCircuit
                from qiskit.circuit import Parameter
                
                # Create a simple parameterized circuit as a fallback
                temp_circuit = QuantumCircuit(self.num_qubits)
                for i in range(self.num_qubits):
                    theta = Parameter(f'θ_{i}')
                    temp_circuit.rx(theta, i)
                for i in range(self.num_qubits - 1):
                    temp_circuit.cx(i, i + 1)
                for i in range(self.num_qubits):
                    phi = Parameter(f'φ_{i}')
                    temp_circuit.rz(phi, i)
                
                ansatz = temp_circuit
            
            logger.info(f"Created {self.ansatz_type} ansatz with {len(ansatz.parameters)} parameters")
            logger.info(f"Ansatz gate counts: {ansatz.count_ops()}")
            
            return ansatz
            
        except Exception as e:
            logger.error(f"Error creating ansatz: {str(e)}")
            # Create a simple fallback ansatz
            from qiskit import QuantumCircuit
            from qiskit.circuit import Parameter
            
            circuit = QuantumCircuit(self.num_qubits)
            params = []
            
            # Create a very simple ansatz with rx and cx gates
            for i in range(self.num_qubits):
                theta = Parameter(f'θ_{i}')
                params.append(theta)
                circuit.rx(theta, i)
            
            for i in range(self.num_qubits - 1):
                circuit.cx(i, i + 1)
            
            logger.info(f"Created fallback ansatz with {len(params)} parameters")
            return circuit
    
    def callback_vqe(self, eval_count, parameters, mean, std, *args, **kwargs):
        """Callback function to track VQE progress with improved error handling."""
        try:
            # Convert std to a float if it's not a numeric type
            if isinstance(std, dict) or not isinstance(std, (int, float, complex, np.number)):
                logger.warning(f"Received non-numeric std value: {type(std)}. Converting to 0.0")
                std_value = 0.0
            else:
                std_value = float(std)
            
            # Safely append to optimization history with proper numeric types
            self.optimization_history.append((
                int(eval_count) if isinstance(eval_count, (int, float)) else eval_count,
                float(mean) if isinstance(mean, (int, float, complex, np.number)) else mean,
                std_value
            ))
            
            # Get system type
            system_type = 'Anti-HeH+' if self.molecular_system.is_anti_matter else 'HeH+'
            
            # Log using safe string formatting
            energy_str = f"{mean:.6f}" if isinstance(mean, (int, float, complex, np.number)) else str(mean)
            std_str = f"{std_value:.6f}"
            logger.info(f"{system_type} - VQE Iteration {eval_count}: Energy = {energy_str} ± {std_str} Hartree")
            
            # Safely bind parameters to circuit
            try:
                # Handle different parameter formats
                if hasattr(parameters, 'values'):  # If parameters is a dictionary-like object
                    bound_circuit = self.ansatz.assign_parameters(list(parameters.values()))
                else:  # If parameters is already a list or array
                    bound_circuit = self.ansatz.assign_parameters(parameters)
                
                self.circuit_history.append((eval_count, bound_circuit))
            except Exception as e:
                logger.warning(f"Failed to bind parameters: {str(e)}")
            
            # Plot progress every 10 iterations with error handling
            if eval_count % 10 == 0:
                try:
                    self.plot_optimization_progress(eval_count)
                except Exception as e:
                    logger.warning(f"Failed to plot optimization progress: {str(e)}")
        
        except Exception as e:
            logger.warning(f"Error in VQE callback: {str(e)}")
            # Don't re-raise the exception to prevent interrupting VQE
    
    def plot_optimization_progress(self, iteration):
        """Plot optimization progress with robust error handling."""
        system_type = 'Anti-HeH+' if self.molecular_system.is_anti_matter else 'HeH+'
        
        try:
            # Plot optimization history
            plt.figure(figsize=(10, 6))
            
            # Extract data with validation
            iterations = []
            energies = []
            errors = []
            
            for item in self.optimization_history:
                try:
                    iter_val, energy_val, error_val = item
                    
                    # Ensure all values are numeric
                    if isinstance(iter_val, (int, float, np.number)) and \
                       isinstance(energy_val, (int, float, complex, np.number)) and \
                       isinstance(error_val, (int, float, complex, np.number)):
                        iterations.append(int(iter_val))
                        energies.append(float(energy_val.real if hasattr(energy_val, 'real') else energy_val))
                        errors.append(float(error_val.real if hasattr(error_val, 'real') else error_val))
                except (ValueError, TypeError, IndexError) as e:
                    logger.warning(f"Skipping invalid optimization history item: {item}. Error: {e}")
            
            if not iterations:
                logger.warning("No valid optimization history to plot")
                return
            
            # Create plot with errorbar
            plt.errorbar(iterations, energies, yerr=errors, fmt='o-')
            plt.axhline(y=self.classical_energy, color='r', linestyle='--', 
                       label=f'Classical: {self.classical_energy:.6f}')
            plt.xlabel('Iteration')
            plt.ylabel('Energy (Hartree)')
            plt.title(f'{system_type} VQE Optimization Progress')
            plt.legend()
            plt.grid(True)
            
            # Save figure with robust error handling
            try:
                filename = f'vqe_progress_{system_type.lower().replace("-", "_")}_{iteration}.png'
                plt.savefig(filename)
                logger.info(f"Saved optimization plot to {filename}")
            except Exception as e:
                logger.warning(f"Failed to save plot: {e}")
            finally:
                plt.close()
                
        except Exception as e:
            logger.warning(f"Failed to plot optimization progress: {e}")
            try:
                plt.close()  # Always try to close the figure
            except:
                pass
    
    def run_vqe(self):
        """Run VQE algorithm."""
        logger.info("Starting VQE algorithm...")
        start_time = time.time()
        
        try:
            # Check if we're using a local simulator fallback
            if hasattr(self, 'use_local_simulator') and self.use_local_simulator:
                # Use local simulator with V2 interface
                logger.info("Using local simulator with V2 interface for VQE")
                
                try:
                    # Try to import the V2 StatevectorEstimator (preferred)
                    from qiskit.primitives import StatevectorEstimator as LocalEstimatorV2
                    local_estimator = LocalEstimatorV2()
                    logger.info("Using StatevectorEstimator (V2)")
                except ImportError:
                    # Fall back to older estimator with warning
                    from qiskit.primitives import Estimator as LocalEstimator
                    local_estimator = LocalEstimator()
                    logger.warning("Using deprecated Estimator. Consider updating to Qiskit 1.2+ with V2 primitives.")
                
                # Create a more robust callback function that can handle different parameter formats
                def callback_wrapper(eval_count, parameters, mean, std, *args, **kwargs):
                    """Wrapper for the callback function to handle different parameter formats."""
                    try:
                        self.callback_vqe(eval_count, parameters, mean, std)
                    except Exception as e:
                        logger.warning(f"Callback error: {str(e)}")
                
                # Use appropriate VQE implementation based on available imports
                try:
                    # Try to import and use the latest VQE implementation (post Qiskit 1.0)
                    from qiskit_algorithms.minimum_eigensolvers import VQE as LocalVQE
                    vqe = LocalVQE(
                        ansatz=self.ansatz,
                        optimizer=self.optimizer,
                        estimator=local_estimator,
                        callback=callback_wrapper
                    )
                except ImportError:
                    # Fall back to older VQE implementation
                    from qiskit_algorithms import VQE as OldVQE
                    vqe = OldVQE(
                        ansatz=self.ansatz,
                        optimizer=self.optimizer,
                        callback=callback_wrapper,
                        quantum_instance=None  # Will use estimator instead
                    )
                    vqe.estimator = local_estimator
                    
                # Run VQE
                result = vqe.compute_minimum_eigenvalue(self.qubit_op)
                self.vqe_result = result
                self.energy = result.eigenvalue.real
            else:
                # Use IBM Quantum Runtime
                logger.info("Using IBM Quantum Runtime for VQE")
                
                # Create custom VQE algorithm for RuntimeEstimator
                class RuntimeVQE:
                    def __init__(self, estimator, ansatz, optimizer, callback=None):
                        self.estimator = estimator
                        self.ansatz = ansatz
                        self.optimizer = optimizer
                        self.callback = callback
                        self.iteration = 0
                        self.best_energy = float('inf')
                        self.best_params = None
                    
                    def compute_expectation(self, parameters):
                        """Compute expectation value with improved transpilation handling."""
                        # Bind parameters to the circuit
                        bound_circuit = self.ansatz.assign_parameters(parameters)
                        
                        try:
                            # Import necessary transpilation tools
                            from qiskit import transpile
                            from qiskit.transpiler import PassManager
                            from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
                            from qiskit.circuit.library import UGate, U3Gate
                            
                            # Get backend to determine supported instructions
                            if hasattr(self.estimator, 'session') and hasattr(self.estimator.session, 'backend'):
                                backend = self.estimator.session.backend
                                
                                # Get target information for strict transpilation
                                target = backend.target
                                logger.info(f"Backend target: {backend.name}")
                                
                                # Generate pass manager with target-aware settings
                                pass_manager = generate_preset_pass_manager(
                                    optimization_level=3,
                                    target=target,  # Use target instead of basis_gates
                                    seed_transpiler=42
                                )
                                
                                # Transpile circuit to target instruction set
                                logger.info("Transpiling circuit to target instruction set...")
                                transpiled_circuit = transpile(
                                    bound_circuit,
                                    backend=backend,
                                    optimization_level=3,
                                    seed_transpiler=42
                                )
                                
                                # Verify circuit is compatible with target
                                from qiskit.transpiler.exceptions import TranspilerError
                                from qiskit_ibm_runtime.utils.validations import validate_isa_circuits
                                
                                try:
                                    validate_isa_circuits([transpiled_circuit], target)
                                    logger.info("Circuit successfully validated against target")
                                except Exception as e:
                                    logger.warning(f"Circuit validation failed: {str(e)}")
                                    # Try more aggressive transpilation if validation fails
                                    logger.info("Attempting more restrictive transpilation...")
                                    transpiled_circuit = transpile(
                                        bound_circuit,
                                        backend=backend,
                                        optimization_level=3,
                                        layout_method='sabre',
                                        routing_method='sabre',
                                        seed_transpiler=42
                                    )
                                    
                            else:
                                # If we can't access the backend, use a default transpilation
                                logger.warning("No backend available, using default transpilation")
                                transpiled_circuit = transpile(
                                    bound_circuit,
                                    basis_gates=['sx', 'rz', 'cx'],  # Modern IBM basis gates
                                    optimization_level=3,
                                    seed_transpiler=42
                                )
                            
                            # Log circuit statistics for debugging
                            logger.info(f"Transpiled circuit depth: {transpiled_circuit.depth()}")
                            logger.info(f"Gate counts: {transpiled_circuit.count_ops()}")
                            
                            # Run the job with transpiled circuit
                            job = self.estimator.run([(transpiled_circuit, self.qubit_op)])
                            result = job.result()
                            
                            # Get expectation value and standard deviation
                            energy = result.values[0]
                            std_dev = result.metadata[0].get('variance', 0) ** 0.5
                            
                            # Update best energy if improved
                            if energy < self.best_energy:
                                self.best_energy = energy
                                self.best_params = parameters
                            
                            # Call callback if provided
                            if self.callback:
                                try:
                                    self.callback(self.iteration, parameters, energy, std_dev)
                                except Exception as e:
                                    logger.warning(f"Callback error: {str(e)}")
                            
                            self.iteration += 1
                            return energy
                            
                        except Exception as e:
                            logger.error(f"Error in compute_expectation: {str(e)}")
                            import traceback
                            logger.error(traceback.format_exc())
                            
                            # If we hit an error, return a high energy value to guide optimizer away
                            # from this point, but not so high that it disrupts optimization
                            self.iteration += 1
                            return 1000.0  # Return high energy value
                    
                    def compute_minimum_eigenvalue(self, operator):
                        self.qubit_op = operator
                        
                        # Get initial parameters
                        initial_params = np.random.random(self.ansatz.num_parameters)
                        
                        # Run optimization
                        result = self.optimizer.minimize(
                            fun=self.compute_expectation,
                            x0=initial_params
                        )
                        
                        # Create a result object similar to Qiskit's VQE
                        from collections import namedtuple
                        VQEResult = namedtuple('VQEResult', ['eigenvalue', 'optimal_point'])
                        
                        # Use best found parameters
                        final_params = result.x if self.best_params is None else self.best_params
                        
                        return VQEResult(
                            eigenvalue=self.best_energy,
                            optimal_point=final_params
                        )
                
                # Create and run VQE
                runtime_vqe = RuntimeVQE(
                    estimator=self.estimator,
                    ansatz=self.ansatz,
                    optimizer=self.optimizer,
                    callback=self.callback_vqe
                )
                
                result = runtime_vqe.compute_minimum_eigenvalue(self.qubit_op)
                self.vqe_result = result
                self.energy = result.eigenvalue
            
            elapsed_time = time.time() - start_time
            logger.info(f"VQE completed in {elapsed_time:.2f} seconds")
            logger.info(f"Quantum energy: {self.energy:.6f} Hartree")
            logger.info(f"Classical energy: {self.classical_energy:.6f} Hartree")
            logger.info(f"Energy difference: {abs(self.energy - self.classical_energy):.6f} Hartree")
            
            # Close session if using IBM Quantum
            if self.session is not None:
                self.session.close()
            
            return self.vqe_result
        except Exception as e:
            logger.error(f"VQE failed: {str(e)}")
            import traceback
            logger.error(traceback.format_exc())
            
            # Close session if using IBM Quantum
            if self.session is not None:
                try:
                    self.session.close()
                except:
                    pass
            
            # Try with local simulator if fallback is enabled
            if self.use_simulator_fallback and not hasattr(self, 'use_local_simulator'):
                logger.info("Falling back to local simulator due to error")
                self.use_local_simulator = True
                return self.run_vqe()  # Recursive call with local simulator
            
            return None
    
    def analyze_results(self, save_plots=True):
        """Analyze VQE results with robust error handling."""
        if self.vqe_result is None:
            logger.warning("No VQE results to analyze")
            return None
        
        system_type = 'Anti-HeH+' if self.molecular_system.is_anti_matter else 'HeH+'
        
        try:
            # Plot final optimization history
            plt.figure(figsize=(10, 6))
            
            # Extract data with validation
            iterations = []
            energies = []
            errors = []
            
            for item in self.optimization_history:
                try:
                    iter_val, energy_val, error_val = item
                    
                    # Ensure all values are numeric
                    if isinstance(iter_val, (int, float, np.number)) and \
                       isinstance(energy_val, (int, float, complex, np.number)) and \
                       isinstance(error_val, (int, float, complex, np.number)):
                        iterations.append(int(iter_val))
                        energies.append(float(energy_val.real if hasattr(energy_val, 'real') else energy_val))
                        errors.append(float(error_val.real if hasattr(error_val, 'real') else error_val))
                except (ValueError, TypeError, IndexError) as e:
                    logger.warning(f"Skipping invalid optimization history item: {item}. Error: {e}")
            
            if not iterations:
                logger.warning("No valid optimization history to plot")
                plt.close()
                return self._create_results_summary()
            
            # Create plot with errorbar
            plt.errorbar(iterations, energies, yerr=errors, fmt='o-', label='VQE Energy')
            plt.axhline(y=self.classical_energy, color='r', linestyle='--', 
                       label=f'Classical: {self.classical_energy:.6f}')
            plt.axhline(y=self.energy, color='g', linestyle='--', 
                       label=f'Final VQE: {self.energy:.6f}')
            plt.xlabel('Iteration')
            plt.ylabel('Energy (Hartree)')
            plt.title(f'{system_type} VQE Optimization Results')
            plt.legend()
            plt.grid(True)
            
            # Save figure with robust error handling
            if save_plots:
                try:
                    filename = f'vqe_final_{system_type.lower().replace("-", "_")}.png'
                    plt.savefig(filename)
                    logger.info(f"Saved optimization plot to {filename}")
                except Exception as e:
                    logger.warning(f"Failed to save plot: {e}")
            
            plt.close()
            
            # Analyze optimal parameters
            param_values = []
            try:
                if hasattr(self.vqe_result, 'optimal_parameters'):
                    optimal_params = self.vqe_result.optimal_parameters
                    param_values = list(optimal_params.values())
                elif hasattr(self.vqe_result, 'optimal_point'):
                    optimal_params = self.vqe_result.optimal_point
                    param_values = list(optimal_params) if hasattr(optimal_params, '__iter__') else [optimal_params]
                else:
                    logger.warning("Could not extract optimal parameters from VQE result")
                    
                if param_values and all(isinstance(p, (int, float, complex, np.number)) for p in param_values):
                    plt.figure(figsize=(10, 6))
                    plt.bar(range(len(param_values)), param_values)
                    plt.xlabel('Parameter Index')
                    plt.ylabel('Value')
                    plt.title(f'{system_type} Optimal VQE Parameters')
                    plt.grid(True)
                    
                    if save_plots:
                        try:
                            filename = f'vqe_parameters_{system_type.lower().replace("-", "_")}.png'
                            plt.savefig(filename)
                            logger.info(f"Saved parameters plot to {filename}")
                        except Exception as e:
                            logger.warning(f"Failed to save parameters plot: {e}")
                    
                    plt.close()
            except Exception as e:
                logger.warning(f"Error analyzing optimal parameters: {e}")
                try:
                    plt.close()
                except:
                    pass
            
            return self._create_results_summary()
            
        except Exception as e:
            logger.warning(f"Error in analyze_results: {e}")
            import traceback
            logger.warning(traceback.format_exc())
            try:
                plt.close()
            except:
                pass
            
            return self._create_results_summary()
    
    def _create_results_summary(self):
        """Create a summary of results with error handling."""
        try:
            system_type = 'Anti-HeH+' if self.molecular_system.is_anti_matter else 'HeH+'
            
            # Calculate relative error safely
            if self.classical_energy != 0:
                relative_error = abs(self.energy - self.classical_energy) / abs(self.classical_energy) * 100
            else:
                relative_error = float('nan')
            
            # Count valid iterations
            valid_iterations = sum(1 for item in self.optimization_history 
                                 if isinstance(item, tuple) and len(item) == 3)
            
            # Create a summary of results
            results_summary = {
                'system_type': system_type,
                'bond_distance': float(self.molecular_system.bond_distance),
                'classical_energy': float(self.classical_energy),
                'quantum_energy': float(self.energy) if isinstance(self.energy, (int, float, complex, np.number)) else None,
                'energy_difference': float(abs(self.energy - self.classical_energy)) if isinstance(self.energy, (int, float, complex, np.number)) else None,
                'relative_error': float(relative_error) if not np.isnan(relative_error) else None,
                'iterations': valid_iterations,
                'backend': str(self.backend_name) if self.backend_name else "local_simulator",
                'shots': int(self.shots),
                'resilience_level': int(self.resilience_level),
                'optimizer': str(self.optimizer_name),
                'ansatz': str(self.ansatz_type),
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
            return results_summary
        except Exception as e:
            logger.warning(f"Error creating results summary: {e}")
            # Return a minimal summary
            return {
                'system_type': 'Anti-HeH+' if self.molecular_system.is_anti_matter else 'HeH+',
                'error': str(e),
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }


def run_comparative_quantum_analysis(bond_distances, 
                                    service, 
                                    backend_name=None,
                                    resilience_level=2,
                                    shots=8192):
    """
    Run quantum simulations for both anti-HeH+ and HeH+ across different bond distances.
    
    Parameters:
    bond_distances (list): List of bond distances to analyze
    service (QiskitRuntimeService): IBM Quantum Runtime service
    backend_name (str): Name of the IBM backend to use
    resilience_level (int): Level of error mitigation
    shots (int): Number of shots per circuit
    
    Returns:
    dict: Comparative results
    """
    results = {
        'distances': bond_distances,
        'anti_matter': {
            'classical_energies': [],
            'quantum_energies': [],
            'errors': []
        },
        'normal_matter': {
            'classical_energies': [],
            'quantum_energies': [],
            'errors': []
        },
        'summaries': []
    }
    
    for dist in bond_distances:
        logger.info(f"===== Bond distance: {dist:.2f} Bohr =====")
        
        # Anti-matter calculation
        logger.info("Computing anti-HeH+...")
        anti_system = AntiMatterMolecularSystem(
            is_anti_matter=True, 
            bond_distance=dist,
            expanded_basis=True
        )
        anti_system.compute_integrals()
        anti_system.map_to_qubit_hamiltonian()
        anti_classical = anti_system.solve_classical()
        
        anti_quantum = QuantumRuntimeSimulation(
            anti_system,
            service=service,
            backend_name=backend_name,
            resilience_level=resilience_level,
            shots=shots
        )
        anti_result = anti_quantum.run_vqe()
        if anti_result:
            anti_summary = anti_quantum.analyze_results()
        else:
            logger.error("Anti-HeH+ VQE failed. Skipping this distance.")
            continue
        
        # Normal matter calculation
        logger.info("Computing HeH+...")
        normal_system = AntiMatterMolecularSystem(
            is_anti_matter=False, 
            bond_distance=dist,
            expanded_basis=True
        )
        normal_system.compute_integrals()
        normal_system.map_to_qubit_hamiltonian()
        normal_classical = normal_system.solve_classical()
        
        normal_quantum = QuantumRuntimeSimulation(
            normal_system,
            service=service,
            backend_name=backend_name,
            resilience_level=resilience_level,
            shots=shots
        )
        normal_result = normal_quantum.run_vqe()
        if normal_result:
            normal_summary = normal_quantum.analyze_results()
        else:
            logger.error("HeH+ VQE failed. Skipping this distance.")
            continue
        
        # Store results
        results['anti_matter']['classical_energies'].append(anti_classical)
        results['anti_matter']['quantum_energies'].append(anti_quantum.energy)
        results['anti_matter']['errors'].append(abs(anti_quantum.energy - anti_classical))
        
        results['normal_matter']['classical_energies'].append(normal_classical)
        results['normal_matter']['quantum_energies'].append(normal_quantum.energy)
        results['normal_matter']['errors'].append(abs(normal_quantum.energy - normal_classical))
        
        results['summaries'].append({
            'distance': dist,
            'anti_matter': anti_summary,
            'normal_matter': normal_summary
        })
        
        # Plot comparative results for this distance
        try:
            plt.figure(figsize=(12, 6))
            plt.title(f'Comparison at Bond Distance {dist:.2f} Bohr')
            
            # Set positions for the bars
            x = np.array([0, 1, 3, 4])
            width = 0.35
            
            # Create bars
            plt.bar(x[0], anti_classical, width, color='blue', alpha=0.5, label='Anti-HeH+ Classical')
            plt.bar(x[1], anti_quantum.energy, width, color='blue', label='Anti-HeH+ Quantum')
            plt.bar(x[2], normal_classical, width, color='red', alpha=0.5, label='HeH+ Classical')
            plt.bar(x[3], normal_quantum.energy, width, color='red', label='HeH+ Quantum')
            
            # Add error bars for quantum results
            plt.errorbar(x[1], anti_quantum.energy, yerr=results['anti_matter']['errors'][-1], 
                        fmt='o', color='black')
            plt.errorbar(x[3], normal_quantum.energy, yerr=results['normal_matter']['errors'][-1], 
                        fmt='o', color='black')
            
            # Add some text for labels, title and axes
            plt.xlabel('System Type')
            plt.ylabel('Energy (Hartree)')
            plt.xticks(x, ['Anti-HeH+\nClassical', 'Anti-HeH+\nQuantum', 'HeH+\nClassical', 'HeH+\nQuantum'])
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.savefig(f'quantum_comparison_distance_{dist:.2f}.png')
            plt.close()
        except Exception as e:
            logger.warning(f"Failed to create distance comparison plot: {e}")
            try:
                plt.close()
            except:
                pass
    
    # Plot overall results
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(results['distances'], results['anti_matter']['classical_energies'], 'b--', 
                label='Anti-HeH+ Classical')
        plt.plot(results['distances'], results['anti_matter']['quantum_energies'], 'bo-', 
                label='Anti-HeH+ Quantum')
        plt.plot(results['distances'], results['normal_matter']['classical_energies'], 'r--', 
                label='HeH+ Classical')
        plt.plot(results['distances'], results['normal_matter']['quantum_energies'], 'ro-', 
                label='HeH+ Quantum')
        
        plt.xlabel('Bond Distance (Bohr)')
        plt.ylabel('Energy (Hartree)')
        plt.title('Quantum vs Classical Energy Comparison')
        plt.legend()
        plt.grid(True)
        
        plt.savefig('quantum_vs_classical_energies.png')
        plt.close()
    except Exception as e:
        logger.warning(f"Failed to create energy comparison plot: {e}")
        try:
            plt.close()
        except:
            pass
    
    # Plot error analysis
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(results['distances'], results['anti_matter']['errors'], 'bo-', 
                label='Anti-HeH+ Error')
        plt.plot(results['distances'], results['normal_matter']['errors'], 'ro-', 
                label='HeH+ Error')
        
        plt.xlabel('Bond Distance (Bohr)')
        plt.ylabel('Absolute Error (Hartree)')
        plt.title('Quantum Simulation Error Analysis')
        plt.legend()
        plt.yscale('log')  # Log scale to better visualize errors
        plt.grid(True)
        
        plt.savefig('quantum_error_analysis.png')
        plt.close()
    except Exception as e:
        logger.warning(f"Failed to create error analysis plot: {e}")
        try:
            plt.close()
        except:
            pass
    
    return results


def study_error_mitigation_impact(bond_distance=1.5, service=None):
    """
    Study the impact of different error mitigation levels with improved parameters.
    
    Parameters:
    bond_distance (float): Bond distance to analyze
    service (QiskitRuntimeService): IBM Quantum Runtime service
    
    Returns:
    dict: Results with different error mitigation settings
    """
    logger.info(f"===== Studying error mitigation impact at bond distance {bond_distance} Bohr =====")
    
    # FIX: Define error mitigation levels with better descriptions
    error_settings = [
        {"name": "No Error Mitigation", "level": 0, "description": "Raw quantum results without mitigation"},
        {"name": "Basic Error Mitigation", "level": 1, "description": "Linear noise extrapolation"},
        {"name": "Advanced Error Mitigation", "level": 2, "description": "Quadratic noise extrapolation with reduced amplifier"}
    ]
    
    results = {
        'anti_matter': {setting['name']: {} for setting in error_settings},
        'normal_matter': {setting['name']: {} for setting in error_settings}
    }
    
    # Initialize systems once with fixed implementation
    anti_system = AntiMatterMolecularSystem(
        is_anti_matter=True, 
        bond_distance=bond_distance,
        expanded_basis=True
    )
    anti_system.compute_integrals()
    anti_system.map_to_qubit_hamiltonian()
    anti_classical = anti_system.solve_classical()
    
    normal_system = AntiMatterMolecularSystem(
        is_anti_matter=False, 
        bond_distance=bond_distance,
        expanded_basis=True
    )
    normal_system.compute_integrals()
    normal_system.map_to_qubit_hamiltonian()
    normal_classical = normal_system.solve_classical()
    
    # Run quantum simulation with different error mitigation settings
    for setting in error_settings:
        logger.info(f"Testing: {setting['name']} - {setting['description']}")
        
        # Anti-matter
        anti_quantum = QuantumRuntimeSimulation(
            anti_system,
            service=service,
            resilience_level=setting['level'],
            shots=8192
        )
        
        anti_result = anti_quantum.run_vqe()
        
        # Normal matter
        normal_quantum = QuantumRuntimeSimulation(
            normal_system,
            service=service,
            resilience_level=setting['level'],
            shots=8192
        )
            
        normal_result = normal_quantum.run_vqe()
        
        # Store results with improved error calculations
        if anti_result:
            anti_error = abs(anti_quantum.energy - anti_classical)
            # Prevent division by zero or very small numbers
            anti_relative_error = (anti_error / abs(anti_classical) * 100) if abs(anti_classical) > 1e-6 else float('inf')
            
            results['anti_matter'][setting['name']] = {
                'classical_energy': anti_classical,
                'quantum_energy': anti_quantum.energy,
                'error': anti_error,
                'relative_error': anti_relative_error,
                'optimization_history': anti_quantum.optimization_history
            }
        
        if normal_result:
            normal_error = abs(normal_quantum.energy - normal_classical)
            # Prevent division by zero or very small numbers
            normal_relative_error = (normal_error / abs(normal_classical) * 100) if abs(normal_classical) > 1e-6 else float('inf')
            
            results['normal_matter'][setting['name']] = {
                'classical_energy': normal_classical,
                'quantum_energy': normal_quantum.energy,
                'error': normal_error,
                'relative_error': normal_relative_error,
                'optimization_history': normal_quantum.optimization_history
            }
    
    # Plot error comparison
    try:
        plt.figure(figsize=(12, 6))
        x = np.arange(len(error_settings))
        width = 0.35
        
        # Anti-matter errors
        anti_errors = [results['anti_matter'][setting['name']].get('relative_error', 0) 
                     for setting in error_settings if setting['name'] in results['anti_matter']]
        normal_errors = [results['normal_matter'][setting['name']].get('relative_error', 0) 
                        for setting in error_settings if setting['name'] in results['normal_matter']]
        
        # Get valid setting names (where we have data)
        valid_settings = [setting['name'] for setting in error_settings 
                         if setting['name'] in results['anti_matter'] and setting['name'] in results['normal_matter']]
        
        plt.bar(x[:len(anti_errors)] - width/2, anti_errors, width, color='blue', label='Anti-HeH+')
        plt.bar(x[:len(normal_errors)] + width/2, normal_errors, width, color='red', label='HeH+')
        
        plt.xlabel('Error Mitigation Setting')
        plt.ylabel('Relative Error (%)')
        plt.title('Impact of Error Mitigation Techniques')
        plt.xticks(x[:len(valid_settings)], valid_settings, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()
        plt.grid(True, alpha=0.3)
        
        plt.savefig('error_mitigation_impact.png')
        plt.close()
    except Exception as e:
        logger.warning(f"Failed to create error mitigation impact plot: {e}")
        try:
            plt.close()
        except:
            pass
    
    return results


def main():
    """Main function to run the anti-matter HeH+ quantum simulation."""
    try:
        logger.info("======= Starting Anti-HeH+ Quantum Research =======")
        
        # Initialize IBM Quantum service directly
        logger.info("Initializing IBM Quantum service")
        service = QiskitRuntimeService("ibm_quantum", "787334d222f9a69a678a3643d6fe8f879db106b442f89fdbca86006a018ff9f6cb3a4946747dccac55aff79ef0fd36e0eeedf9e6dd07ff86763b401722d99e00")
        
        # List available backends
        backends = service.backends()
        backend_names = [b.name for b in backends]
        logger.info(f"Available backends: {backend_names}")
        
        # Let user select backend
        print("\nAvailable backends:")
        real_hw_backends = []
        simulators = []
        
        for b in backends:
            is_simulator = getattr(b.configuration(), 'simulator', False)
            if is_simulator:
                simulators.append(b.name)
            else:
                real_hw_backends.append(b.name)
        
        print("Real hardware:")
        for i, name in enumerate(real_hw_backends):
            print(f"  {i+1}. {name}")
        
        print("\nSimulators:")
        for i, name in enumerate(simulators):
            print(f"  {i+len(real_hw_backends)+1}. {name}")
        
        try:
            choice = int(input("\nSelect backend number (or 0 for auto-select): "))
            if choice == 0:
                backend_name = None  # Auto-select
            else:
                all_backends = real_hw_backends + simulators
                backend_name = all_backends[choice-1]
            logger.info(f"Selected backend: {backend_name if backend_name else 'Auto-select'}")
        except (ValueError, IndexError):
            logger.info("Invalid selection. Using auto-select.")
            backend_name = None
        
        # Define parameters for the study
        bond_distances = [0.8, 1.5, 2.5]  # Limited set for quantum simulation
        resilience_level = 2  # Maximum error mitigation
        shots = 8192  # Number of shots per circuit
        
        # Run comparative analysis
        logger.info("Running comparative analysis...")
        comparison_results = run_comparative_quantum_analysis(
            bond_distances=bond_distances,
            service=service,
            backend_name=backend_name,
            resilience_level=resilience_level,
            shots=shots
        )
        
        # Study error mitigation impact
        logger.info("Studying error mitigation impact...")
        error_mit_results = study_error_mitigation_impact(
            bond_distance=1.5,
            service=service
        )
        
        # Save combined results
        results = {
            'comparative_analysis': comparison_results,
            'error_mitigation_study': error_mit_results,
            'params': {
                'bond_distances': bond_distances,
                'backend': backend_name,
                'resilience_level': resilience_level,
                'shots': shots
            },
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        # Save to JSON file with robust error handling
        try:
            results_file = f"anti_heh_quantum_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            
            # Custom JSON serializer to handle special types
            def json_serializer(obj):
                if isinstance(obj, (complex, np.number, np.ndarray)):
                    return str(obj)
                # Handle other special types
                try:
                    return str(obj)
                except:
                    return "unserializable_object"
            
            with open(results_file, 'w') as f:
                json.dump(results, f, default=json_serializer, indent=2)
            
            logger.info(f"Results saved to {results_file}")
        except Exception as e:
            logger.error(f"Failed to save results to JSON: {str(e)}")
            # Try saving to a simpler format if JSON fails
            try:
                backup_file = f"anti_heh_quantum_results_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
                with open(backup_file, 'w') as f:
                    f.write(str(results))
                logger.info(f"Backup results saved to {backup_file}")
            except:
                logger.error("Could not save results in any format")
        
        logger.info("======= Anti-HeH+ Quantum Research Completed =======")
        
    except Exception as e:
        logger.error(f"Error in quantum research: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())


# Main execution
if __name__ == "__main__":
    main()

2025-03-09 11:00:00,080 - INFO - Pass: UnrollCustomDefinitions - 0.28849 (ms)
2025-03-09 11:00:00,085 - INFO - Pass: BasisTranslator - 0.74482 (ms)
2025-03-09 11:00:01,740 - INFO - ======= Starting Anti-HeH+ Quantum Research =======
2025-03-09 11:00:01,741 - INFO - Initializing IBM Quantum service
qiskit_runtime_service.__init__:INFO:2025-03-09 11:00:06,628: Default instance: ibm-q/open/main
2025-03-09 11:00:10,610 - INFO - Available backends: ['ibm_brisbane', 'ibm_kyiv', 'ibm_sherbrooke']



Available backends:
Real hardware:
  1. ibm_brisbane
  2. ibm_kyiv
  3. ibm_sherbrooke

Simulators:


2025-03-09 11:00:10,853 - INFO - Invalid selection. Using auto-select.
2025-03-09 11:00:10,853 - INFO - Running comparative analysis...
2025-03-09 11:00:10,854 - INFO - ===== Bond distance: 0.80 Bohr =====
2025-03-09 11:00:10,855 - INFO - Computing anti-HeH+...
2025-03-09 11:00:10,855 - INFO - Computing integrals for anti-matter HeH+ at 0.8 Bohr
2025-03-09 11:00:25,013 - INFO - One-electron Hamiltonian diagonal: min=-2.439426, max=-1.164758
2025-03-09 11:00:25,014 - INFO - Two-electron integral stats: min=0.000000, max=0.000000
2025-03-09 11:00:25,015 - INFO - Integral computation completed in 14.16 seconds
2025-03-09 11:00:25,016 - INFO - Setting up anti-matter HeH+ with (2,1) particles
2025-03-09 11:00:25,047 - INFO - Mapped to 8-qubit Hamiltonian using jordan_wigner mapping
2025-03-09 11:00:25,048 - INFO - Number of Pauli terms: 33
2025-03-09 11:00:28,581 - INFO - Classical solver energy: -20.629928 Hartree
2025-03-09 11:00:28,582 - INFO - Lowest eigenvalues: [-10.31496405   0.40753

KeyboardInterrupt: 